# Import learning data

In [1]:
import pandas as pd

training_data1_path = "../../learning_data/data/win_rate/turn/50000-data-1000-simulation-2-players-win-rate-data.csv"
training_data2_path = "../../learning_data/data/win_rate/turn/100000-data-1000-simulation-2-players-win-rate-data.csv"
test_data_path = "../../learning_data/data/win_rate/turn/10000-data-1000-simulation-2-players-win-rate-data.csv"
train1_df = pd.read_csv(training_data1_path)
train2_df = pd.read_csv(training_data2_path)
train_df = train1_df.append(train2_df, ignore_index=True)
test_df = pd.read_csv(test_data_path)

# About learning data

In [2]:
print train_df.shape, test_df.shape

(150000, 13) (10000, 13)


In [3]:
train_df.head()

,hole1,hole2,community1,community2,community3,community4,win_rate,hole1_id,hole2_id,community1_id,community2_id,community3_id,community4_id
0,S8,C7,D8,S6,H8,H4,0.969,47,7,21,45,34,30
1,H9,SA,HJ,D7,HK,S5,0.459,35,40,37,20,39,44
2,SK,D2,H2,CT,HJ,D9,0.531,52,15,28,10,37,22
3,D3,C2,S8,CJ,H2,C5,0.520,16,2,47,11,28,5
4,S4,C5,CJ,HJ,H2,H6,0.591,43,5,11,37,28,32


In [4]:
train_df.describe()

,win_rate,hole1_id,hole2_id,community1_id,community2_id,community3_id,community4_id
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000
mean,0.585071,26.521973,26.477533,26.488147,26.463247,26.513620,26.509133
std,0.196277,15.007795,15.004448,15.009793,14.992676,14.978887,15.002317
min,0.107000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,0.430000,14.000000,13.000000,14.000000,13.000000,14.000000,14.000000
50%,0.582000,27.000000,27.000000,26.000000,26.000000,27.000000,27.000000
75%,0.722000,40.000000,39.000000,39.000000,39.000000,39.000000,39.000000
max,1.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000


In [5]:
test_df.describe()

,win_rate,hole1_id,hole2_id,community1_id,community2_id,community3_id,community4_id
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000
mean,0.584710,26.443700,26.590600,26.589900,26.45000,26.648700,26.451400
std,0.196106,15.064466,14.913062,14.970869,14.87904,14.876902,15.142436
min,0.145000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000
25%,0.428000,13.000000,14.000000,14.000000,14.00000,14.000000,13.000000
50%,0.583000,26.000000,27.000000,27.000000,27.00000,27.000000,26.000000
75%,0.721000,39.000000,39.000000,39.000000,39.00000,40.000000,40.000000
max,1.000000,52.000000,52.000000,52.000000,52.00000,52.000000,52.000000


# Data Processing

## card id -> 1-hot formated vector

In [6]:
import numpy as np
from pypokerengine.engine.card import Card

fetch_hole = lambda row: [row[key] for key in ['hole1_id', 'hole2_id']]
fetch_community = lambda row: [row[key] for key in ['community1_id', 'community2_id', 'community3_id', 'community4_id']]
gen_card_rank_vec = lambda card: [1 if r == card.rank else 0 for r in range(2,15)]
gen_card_suit_vec = lambda card: [1 if s == card.suit else 0 for s in [Card.CLUB, Card.DIAMOND, Card.HEART, Card.SPADE]]
gen_card_vec = lambda card: gen_card_rank_vec(card) + gen_card_suit_vec(card)
gen_img = lambda zipped: [gen_card_vec(Card.from_id(card_id)) for card_id in zipped[0]+zipped[1]]

train_hole = train_df.apply(lambda row: fetch_hole(row), axis=1)
train_community = train_df.apply(lambda row: fetch_community(row), axis=1)
train_df["onehot"] = map(gen_img, zip(train_hole, train_community))

test_hole = test_df.apply(lambda row: fetch_hole(row), axis=1)
test_community = test_df.apply(lambda row: fetch_community(row), axis=1)
test_df["onehot"] = map(gen_img, zip(test_hole, test_community))

## Format data (pandas.df -> numpy.ndarray)

In [7]:
wrap = lambda lst: np.array(lst)
to_ndarray = lambda X: wrap([wrap(x) for x in X])
train_x, train_y = [to_ndarray(array) for array in [train_df["onehot"].values, train_df["win_rate"].values]]
test_x, test_y = [to_ndarray(array) for array in [test_df["onehot"].values, test_df["win_rate"].values]]
train_x, test_x = [X.reshape(X.shape[0], 1, X.shape[1], X.shape[2]) for X in [train_x, test_x]]
print "shape of training data => x: %s, y: %s" % (train_x.shape, train_y.shape)
print "shape of test data => x: %s, y: %s" % (test_x.shape, test_y.shape)

shape of training data => x: (150000, 1, 6, 17), y: (150000,)
shape of test data => x: (10000, 1, 6, 17), y: (10000,)


# Create model

In [8]:
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.core import Dense, Activation, Dropout, Flatten

model = Sequential()

model.add(Convolution2D(256, 3, 3, border_mode="same", input_shape=(1, 6, 17)))
model.add(Activation("relu"))
model.add(Convolution2D(256, 3, 3, border_mode="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(256, 3, 3, border_mode="same", input_shape=(1, 6, 17)))
model.add(Activation("relu"))
model.add(Convolution2D(256, 3, 3, border_mode="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(1))
model.compile(loss="mse",  optimizer="adadelta")

Using Theano backend.


# Train or Load existing model

In [9]:
import os
model_weight_file_path = "./resource/turn_cnn_model_weights.h5"
if os.path.exists(model_weight_file_path):
    model.load_weights(model_weight_file_path)
else:
    history = model.fit(train_x, train_y, batch_size=128, nb_epoch=1, validation_split=0.1, verbose=2)

# Check model performance

## Visualize loss transition

In [10]:
%matplotlib inline
import seaborn
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
for idx, key in enumerate(history.history, start=1):
    plt.subplot(2, 1, idx)
    plt.plot(range(len(history.history[key])), history.history[key])
    plt.title(key)
plt.show()

<img src="./resource/turn_cnn_loss_history.png" width=800/>

## Test model performance by MSE 

In [10]:
from sklearn.metrics import mean_squared_error

def print_model_performance(model, train_x, train_y, test_x, test_y):
    print 'MSE on training data = {score}'.format(score=mean_squared_error(model.predict(train_x), train_y))
    print 'MSE on test data = {score}'.format(score=mean_squared_error(model.predict(test_x), test_y))

In [11]:
print_model_performance(model, train_x, train_y, test_x, test_y)

MSE on training data = 0.000162176776933
MSE on test data = 0.000420929524004


## See model prediction on sample data

In [12]:
from pypokerengine.engine.card import Card
C, D, H, S = Card.CLUB, Card.DIAMOND, Card.HEART, Card.SPADE

test_case = [
    [(8, D), (4, C), (13, C), (1, C), (6, C), (2, C), 0.994],
    [(3, D), (3, H), (5, H), (4, D), (5, S), (4, H), 0.11],
    [(1, C), (10, D), (6, H), (9, H), (8, D), (11, C), 0.505]
]

to_id = lambda card: card.to_id()
gen_card_rank_vec = lambda card: [1 if r == card.rank else 0 for r in range(2,15)]
gen_card_suit_vec = lambda card: [1 if s == card.suit else 0 for s in [Card.CLUB, Card.DIAMOND, Card.HEART, Card.SPADE]]
gen_card_vec = lambda card: gen_card_rank_vec(card) + gen_card_suit_vec(card)
gen_img = lambda zipped: [gen_card_vec(Card.from_id(card_id)) for card_id in zipped[0]+zipped[1]]
wrap = lambda lst: np.array(lst)
to_ndarray = lambda X: wrap([wrap(x) for x in X])

for card1, card2, card3, card4, card5, card6, expected in test_case:
    cards = [Card(rank=rank, suit=suit) for rank, suit in [card1, card2, card3, card4, card5, card6]]
    hole = cards[:2]
    community = cards[2:]
    hole_ids = map(to_id, hole)
    community_ids = map(to_id, community)
    x = gen_img((hole_ids, community_ids))
    X = to_ndarray(x)
    X = np.array([X.reshape(1, X.shape[0], X.shape[1])])
    y = model.predict(X)[0][0]
    print "HOLE = [%s, %s], COMMUNITY = [%s, %s, %s, %s] => win_rate = { prediction=%f, expected=%f }" % tuple(map(str, hole) + map(str, community) + [y , expected])

HOLE = [D8, C4], COMMUNITY = [CK, CA, C6, C2] => win_rate = { prediction=1.004189, expected=0.994000 }
HOLE = [D3, H3], COMMUNITY = [H5, D4, S5, H4] => win_rate = { prediction=0.440482, expected=0.110000 }
HOLE = [CA, DT], COMMUNITY = [H6, H9, D8, CJ] => win_rate = { prediction=0.506822, expected=0.505000 }
